# Kohya SDXL Fine‑Tune Notebook
This Colab notebook fine‑tunes an SDXL model with [kohya‑ss/sd‑scripts](https://github.com/kohya-ss/sd-scripts).
_Automatically instrumented for detailed debug logging as requested by Mikey._

### Clone kohya‑ss/sd‑scripts

In [9]:
# Install required libraries
%cd /content
!git clone https://github.com/kohya-ss/sd-scripts.git
%cd sd-scripts
!pip install -r requirements.txt


Cloning into '/content/sd-scripts'...
remote: Enumerating objects: 10050, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 10050 (delta 53), reused 55 (delta 37), pack-reused 9959 (from 4)
Receiving objects: 100% (10050/10050), 11.95 MiB | 14.74 MiB/s, done.
Resolving deltas: 100% (7193/7193), done.
/content/sd-scripts
Obtaining file:///content/sd-scripts
  Preparing metadata (setup.py) ... done
  Running setup.py develop for library


NameError: name 'log' is not defined

## 2. Prepare Google Drive (optional)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


#  Set Training Paths


In [ ]:
import os# Define pathsbase_model_path = "/content/drive/MyDrive/models/homosimileXLPony_v20ILXL.safetensors"diff_model_path = "/content/drive/MyDrive/models/homosimileXLPony_v20ILXL_diffusers"dataset_dir = "/content/drive/MyDrive/Loras/kmk/dataset"output_dir = "/content/drive/MyDrive/Loras/kmk/output"logging_dir = "/content/drive/MyDrive/Loras/kmk/logs"cache_latents_dir = "/content/drive/MyDrive/Loras/kmk/cache_latents"# Make sure all folders existos.makedirs(output_dir, exist_ok=True)os.makedirs(logging_dir, exist_ok=True)os.makedirs(cache_latents_dir, exist_ok=True)

In [ ]:
# Convert checkpoint to Diffusers format!python - <<'PY'
import os
from library import sdxl_model_util

ckpt = base_model_path
output_dir = diff_model_path
os.makedirs(output_dir, exist_ok=True)
print('Converting checkpoint to Diffusers format...')
text_encoder1, text_encoder2, vae, unet, _, _ = sdxl_model_util.load_models_from_sdxl_checkpoint(sdxl_model_util.MODEL_VERSION_SDXL_BASE_V1_0, ckpt, 'cpu')
sdxl_model_util.save_diffusers_checkpoint(output_dir, text_encoder1, text_encoder2, unet, None, vae, use_safetensors=True)
print('Saved Diffusers model to', output_dir)
PY

# ✅  Prepare latents

In [ ]:
# Optional speed boost: Cache latents
!python tools/cache_latents.py \
  --vae batch \
  --batch_size 1 \
  --model_name_or_path="{diff_model_path}" \\n  --data_dir="{dataset_dir}" \
  --output_dir="{cache_latents_dir}"


#✅ start training


In [ ]:
!python sdxl_train_network.py \
  --pretrained_model_name_or_path="{diff_model_path}" \\n  --train_data_dir="{dataset_dir}" \
  --resolution="1024,1024" \
  --output_dir="{output_dir}" \
  --logging_dir="{logging_dir}" \
  --output_name="kmk_sdxl_lora" \
  --network_module=networks.lora \
  --network_dim=256 \
  --network_alpha=128 \
  --network_train_unet_only \
  --learning_rate=1e-4 \
  --lr_scheduler="cosine_with_restarts" \
  --train_batch_size=1 \
  --max_train_steps=1500 \
  --save_every_n_steps=500 \
  --mixed_precision="fp16" \
  --cache_latents \
  --cache_latents_to_disk \
  --cache_latents_dir="{cache_latents_dir}" \
  --caption_extension=".txt" \
  --caption_dropout_rate=0.15 \
  --xformers


#✅ Check outputs

In [ ]:
# Check output folder contents
import os

for f in os.listdir(output_dir):
    print(f)
